In [ ]:
!pwd

/content/drive/MyDrive/nlp_project


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install KRWordRank
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
from collections import Counter
import re

In [ ]:
%cd /content/drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [ ]:
import sys
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
import numpy as np
import pandas as pd

## **데이터 파일 전처리 함수**

In [ ]:
def csv2list(filename) -> list: # csv파일에서 상호명만 분리
  df = pd.read_csv(filename, encoding = 'utf-8')
  df1 = df['상호명']
  restaurant_list = []
  for item in df1:
    restaurant_list.append(item)
  
  return restaurant_list

In [ ]:
def fname_list(list, type) -> list: #대분류 안에 있는 파일 '....txt'형태로 만들기
  fname_list = []
  for name in list:
    txt_name = type+'/'+name+'.txt'
    fname_list.append(txt_name)

  return fname_list

def fname_normlist(list, type) -> list: #대분류 안에 있는 파일 '.._norm.txt'형태로 만들기
  fname_list = []
  for name in list:
    txt_name = type+'/'+name+'_norm.txt'
    fname_list.append(txt_name)

  return fname_list

## **파일 전처리 및 알고리즘 적용 전 사전 준비**

In [ ]:
# 1: 일식, 2: 한식, 3: 양식, 4: 베이커리, 5: 중식, 6: 패스트푸드, 7: 디저트카페
top_keywords_1, top_keywords_2, top_keywords_3, top_keywords_4, top_keywords_5, top_keywords_6, top_keywords_7 = [], [], [], [], [], [], []
restaurant_type = ['일식', '한식', '양식', '베이커리', '중식', '패스트푸드', '디저트카페']

# 대분류 파일 마다 있는 csv 파일 만들기 ex) 일식/강남구_일식.csv
filename_1 = restaurant_type[0]+'/강남구_'+restaurant_type[0]+'.csv' 
filename_2 = restaurant_type[1]+'/강남구_'+restaurant_type[1]+'.csv'
filename_3 = restaurant_type[2]+'/강남구_'+restaurant_type[2]+'.csv'
filename_4 = restaurant_type[3]+'/강남구_'+restaurant_type[3]+'.csv'
filename_5 = restaurant_type[4]+'/강남구_'+restaurant_type[4]+'.csv'
filename_6 = restaurant_type[5]+'/강남구_'+restaurant_type[5]+'.csv'
filename_7 = restaurant_type[6]+'/강남구_'+restaurant_type[6]+'.csv'


# 각 대분류마다 있는 csv파일에서 식당이름만 추출하여 리스트로 만들기 
restaurant_name_list_1 = csv2list(filename_1)
restaurant_name_list_2 = csv2list(filename_2)
restaurant_name_list_3 = csv2list(filename_3)
restaurant_name_list_4 = csv2list(filename_4)
restaurant_name_list_5 = csv2list(filename_5)
restaurant_name_list_6 = csv2list(filename_6)
restaurant_name_list_7 = csv2list(filename_7)

# 각 대분류 안에 있는 식당들의 이름들을 .txt 형태로 만들어주기 ex) 일식/도쿄우동.txt
fnames_1 = fname_list(restaurant_name_list_1, restaurant_type[0])
fnames_2 = fname_list(restaurant_name_list_2, restaurant_type[1])
fnames_3 = fname_list(restaurant_name_list_3, restaurant_type[2])
fnames_4 = fname_list(restaurant_name_list_4, restaurant_type[3])
fnames_5 = fname_list(restaurant_name_list_5, restaurant_type[4])
fnames_6 = fname_list(restaurant_name_list_6, restaurant_type[5])
fnames_7 = fname_list(restaurant_name_list_7, restaurant_type[6])


# 각 대분류 안에 존재하는 식당 개수 출력
n_fnames_1 = len(fnames_1)
n_fnames_2 = len(fnames_2)
n_fnames_3 = len(fnames_3)
n_fnames_4 = len(fnames_4)
n_fnames_5 = len(fnames_5)
n_fnames_6 = len(fnames_6)
n_fnames_7 = len(fnames_7)

# print(len(fnames_2))
print(len(fnames_1),len(fnames_2),len(fnames_3),len(fnames_4),len(fnames_5),len(fnames_6),len(fnames_7))

28 59 44 22 19 36 44


### **KR-WordRank 알고리즘**

In [ ]:
def get_texts_scores(fname) -> tuple:
    # `fname` 파일은 리뷰평과 평점이 \t 으로 구분된 two column tsv 파일입니다.
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]

        if not docs:
            return [], []

        texts, scores = zip(*docs)
        return list(texts), list(scores)

In [ ]:
# #KR-wordrank 알고리즘 테스트(장준원)

# # def kr_wordrank_algorithm(list, fname) -> list:
# #     texts, scores = get_texts_scores(fname)
# #     han = Hannanum()
# #     for i in range(len(texts)):
# #       for j in range(len(han.nouns(texts[i]))):
# #         if han.nouns(texts[i])[j] not in noun_list:
# #           noun_list.append(han.nouns(texts[i])[j])
        
# #     wordrank_extractor = KRWordRank(
# #         min_count=3, max_length=10, verbose=False)
    
# #     beta = 0.85    # PageRank의 decaying factor beta
# #     max_iter = 10

# #     keywords, rank, graph = wordrank_extractor.extract(
# #         texts, beta, max_iter)
    
# #     list.append(
# #         sorted(keywords.items(),
# #                key=lambda x:x[1],
# #                reverse=True)[:100]
# #     )
# #     print('식당')
# #     return list

In [ ]:
# filename = "한식/갈비다움 압구정지점.txt"
# f = open(filename, 'r', encoding='utf-8')
# news = f.read()

# noun = okt.nouns(news)
# for i,v in enumerate(noun):
#   if len(v)<2:
#     noun.pop(i)

# count = Counter(noun)
# f.close()

# noun_list = count.most_common(10)
# print(noun_list)

In [ ]:
# 각 대분류 주요 명사 리스트
noun_list_1, noun_list_2, noun_list_3, noun_list_4, noun_list_5, noun_list_6, noun_list_7 = [], [], [], [], [], [], []
okt = Okt()

def extracting_nounlist(noun_list, filenames) -> list:

  for file in filenames:
    f = open(file, 'r', encoding='utf-8')
    temp_file = f.read()
    noun = okt.nouns(temp_file)

    for i, v in enumerate(noun):
      if len(v) < 2:
        noun.pop(i)

    count = Counter(noun)
    top_ten_count = count.most_common(5)

    for key, counting in top_ten_count:
      noun_list.append([key, counting])
      
#####################################################################
# 오름차순 정렬      
  sorted_list = {}
  for pair in noun_list: # 딕셔너리로 변환해서 value값 합치기
    word = pair[0]
    cnt = pair[1]

    if word in sorted_list:
      sorted_list[word] += cnt
    else:
      sorted_list[word] = cnt

  sorted_list=list(zip(sorted_list.keys(),sorted_list.values()))# 딕셔너리 리스트로 변환

  sorted_list.sort(key=lambda x: x[1],reverse=1)# value값으로 오름차순 정렬

  word_list = []
  for z in range(len(sorted_list)):
    if sorted_list[z][0] == '양도' or sorted_list[z][0] == '양이':
       word_list.append('양') 
    else:
      word_list.append(sorted_list[z][0])
  
  return word_list

In [ ]:
noun_list_1 = extracting_nounlist(noun_list_1, fnames_1)
noun_list_2 = extracting_nounlist(noun_list_2, fnames_2)
noun_list_3 = extracting_nounlist(noun_list_3, fnames_3)
noun_list_4 = extracting_nounlist(noun_list_4, fnames_4)
noun_list_5 = extracting_nounlist(noun_list_5, fnames_5)
noun_list_6 = extracting_nounlist(noun_list_6, fnames_6)
noun_list_7 = extracting_nounlist(noun_list_7, fnames_7)

In [ ]:
# print("noun_list_1 =" , noun_list_1)
# print("noun_list_2 =" , noun_list_2)
# print("noun_list_3 =" , noun_list_3)
# print("noun_list_4 =" , noun_list_4)
# print("noun_list_5 =" , noun_list_5)
# print("noun_list_6 =" , noun_list_6)
# print("noun_list_7 =" , noun_list_7)

In [ ]:
#KR-wordrank 알고리즘
def kr_wordrank_algorithm(list, fname) -> list:
    texts, scores = get_texts_scores(fname)
    texts2 = []
    for i in texts:
      i = re.sub(r'[^0-9가-힣 ]', '', i)
      texts2.append(i)

    wordrank_extractor = KRWordRank(
        min_count=3, max_length=10, verbose=False)
    
    beta = 0.85    # PageRank의 decaying factor beta
    max_iter = 10

    keywords, rank, graph = wordrank_extractor.extract(
        texts2, beta, max_iter)
    
    list.append(
        sorted(keywords.items(),
               key=lambda x:x[1],
               reverse=True)[:100]
    )
    return list

## **각 대분류 마다 KR-wordrank 알고리즘 적용**

In [ ]:
for fname in fnames_1: 
  kr_wordrank_algorithm(top_keywords_1, fname)

for fname in fnames_2: 
  kr_wordrank_algorithm(top_keywords_2, fname)
  
for fname in fnames_3: 
  kr_wordrank_algorithm(top_keywords_3, fname)

for fname in fnames_4: 
  kr_wordrank_algorithm(top_keywords_4, fname)

for fname in fnames_5: 
  kr_wordrank_algorithm(top_keywords_5, fname)

for fname in fnames_6: 
  kr_wordrank_algorithm(top_keywords_6, fname)

for fname in fnames_7: 
  kr_wordrank_algorithm(top_keywords_7, fname)

## **키워드 보여주는 함수**

In [ ]:
def show_keywords(list, n, restaurant_name):
  name = '     '.join(
          ['       %s'         % (restaurant_name[s][:10])
          for s in range(n)])
  
  print(name)
  print()

  for k in range(20):
      message = '  --  '.join(
          ['%8s (%.3f)' % (list[i][k][0],list[i][k][1])
          for i in range(n)])
      
      print(message)

In [ ]:
show_keywords(top_keywords_1, n_fnames_1, restaurant_name_list_1)

       은행골 신사점            압구정하루            갓덴스시 강남점            후레쉬빌 삼성역본점            미미면가            샤블리            감탄성신            이끼롤까스 가로수길            상무초밥 강남역점            스시메이진 강남점            미도인 강남            부타이 1막            정돈 강남점            스시마이우 강남CG            홈수끼 학동점            카메스시            샤브쌈주머니 서울강            온기정            카츠바이콘반            청담돈            마코토 청담점            지구당 선릉역점            호호식당 도산공원            스시롭다            이승재참치            올바른스시 강남역점            야스노야 지로 압구            스시마카세 역삼점

      맛있 (22.327)  --       냉모밀 (16.916)  --        맛있 (16.232)  --        너무 (44.980)  --        맛있 (17.561)  --        너무 (45.682)  --      맛있어요 (40.602)  --        너무 (35.634)  --        맛있 (19.308)  --        초밥 (14.509)  --        맛있 (19.629)  --        맛있 (18.251)  --        맛있 (16.529)  --        너무 (33.177)  --        너무 (44.719)  --        너무 (51.235)  --        맛있 (25.884)  --        맛있 (16.692)  --        맛있 (16.614)  --        너무 (24.434)  --        너무 (35.07

In [ ]:
show_keywords(top_keywords_2, n_fnames_2, restaurant_name_list_2)

       중앙해장            도쿄등심 압구정점            진미평양냉면            정식당            뱃고동            장인닭갈비 강남점            농민백암순대 본점            탕탕집 논현본점            뽕나무쟁이 선릉본점            새벽집 청담동점            리복집            호족반            도쿄등심 청담점            강남 돼지상회 무한            무월식탁            창고43 강남점            삼육가 본점            류몽민 논현본점            하동관 코엑스몰직영            뱅뱅막국수            미사리밀빛초계국수             농민백암순대 본점            참족            강남 곱            도쿄등심 도곡점            강강술래 역삼점            고기꾼김춘배 강남점            두껍삼 역삼직영점            피양콩할마니            곱창고 강남역점            탐라도야지            백억하누 강남본점            까사생갈비            신둥궁감자탕 선릉직            화기애애            청담동 순도리            온돌 더프라임 강남            무한리필 몽블리 강            수라선            장다리곱창            명랑회관 선릉점            이대성의진면목            명동관            진대감 선릉점            로제닭갈비            남다른감자탕 역삼본            밥밥디라라 역삼본점            갈비다움 역삼본점            램브라튼            창고43 삼성점            송포갈비            이층고깃집 강남구청            수묵당     

In [ ]:
show_keywords(top_keywords_3, n_fnames_3, restaurant_name_list_3)

       울프강 스테이크하우            어글리스토브 신논현            바비레드 강남본점            치즈룸 안다즈            페어링룸            어거스트 힐 강남점            다운타우너 청담            마초쉐프 강남본점            스킬렛 스테이크하우            리틀넥 청담            꽁티드툴레아            크라이치즈버거 삼성            어썸로즈            더플레이스 스타필드            브루클린더버거조인트            메리고라운드 스테이            마녀주방 강남점            텍사스데브라질 압구            클랩피자 청담점            파씨오네            미즈컨테이너 강남             감성타코 강남역점            빌즈 강남            BISTROT de            칙피스 신사점            인터컨티넨탈 코엑스            빠레뜨한남 가로수길            카페마마스 코엑스점            로마옥            을지다락 강남            비어룸            달링키친            코벤트가든 선릉점            마켓오 도곡점            볼트스테이크하우스            노모어피자            오스테리아 꼬또 압            빠레뜨한남 강남CG            미도니셰프 강남점            미도니셰프 삼성점            오스틴            코르바니 선릉            나폴리회관            빠비뚜엡

      너무 (115.266)  --        맛있 (51.554)  --        맛있 (24.604)  --        너무 (86.197)  --        너무 (98.583)  --        너무 (78.665)  --

In [ ]:
show_keywords(top_keywords_4, n_fnames_4, restaurant_name_list_4)

       아우어베이커리 도산            아우어베이커리 가로            노아베이커리 압구정            파리바게뜨 양재본점            김영모과자점 강남구            김영모과자점 대치점            파리크라상 삼성역키            파리크라상 도곡점            삼송빵집 신논현점            노아베이커리 GFC            비파티세리            타르틴베이커리 도산            아우어베이커리 싸이            파리바게뜨 대청타워            파리바게뜨 신분당선            파리바게뜨 카페선릉            바게트케이            파리바게뜨 압구정역            베이커리로컬 강남역            밀도 한티역점            파리바게뜨 강남역점            김영모과자점 도곡타

      맛있 (27.323)  --        맛있 (20.912)  --        맛있 (39.660)  --       좋아요 (13.100)  --      맛있어요 (25.557)  --      맛있어요 (20.438)  --      맛있어요 (17.316)  --        맛있 (13.219)  --      맛있어요 (17.208)  --        너무 (28.313)  --        맛있 (20.689)  --      맛있어요 (14.583)  --        맛있 (14.386)  --        좋아 (6.569)  --       좋아요 (9.168)  --       좋아요 (7.386)  --      맛있어요 (18.647)  --       좋아요 (10.579)  --        맛있 (13.339)  --      맛있어요 (13.071)  --       좋아요 (9.265)  --      맛있어요 (27.647)
      너무 (12.428)  --        너무 (12.6

In [ ]:
show_keywords(top_keywords_5, n_fnames_5, restaurant_name_list_5)

       SMC싱푸차이나            팀호완 삼성점            쮸즈            딤딤섬 파르나스몰점            무탄            라공방 강남1호 본            중경마라탕 강남역점            루위            니뽕내뽕 코엑스점            소각            남소관 우육도삭면            공리            인량훠궈            시추안하우스 삼성점            무한숯불양고치&훠궈            파불라            샤오당쟈마라탕 신사            마라공방 역삼점            미엔아이 천등

    맛있어요 (17.840)  --        맛있 (34.147)  --        맛있 (18.762)  --        맛있 (23.741)  --        맛있 (24.851)  --        맛있 (14.312)  --      맛있어요 (16.809)  --        너무 (38.084)  --        맛있 (14.082)  --        너무 (43.138)  --        맛있 (23.356)  --        맛있 (20.394)  --        너무 (24.690)  --        맛있 (17.701)  --       양꼬치 (21.807)  --        너무 (20.355)  --      맛있어요 (17.782)  --      맛있어요 (21.384)  --      맛있어요 (34.079)
      좋아 (14.917)  --        너무 (24.820)  --        너무 (12.485)  --        딤섬 (15.647)  --        너무 (24.446)  --       마라탕 (13.610)  --       마라탕 (13.706)  --        맛있 (25.115)  --        너무 (10.124)  --      맛있어요 (3

In [ ]:
show_keywords(top_keywords_6, n_fnames_6, restaurant_name_list_6)

       쉐이크쉑 강남점            에그슬럿 코엑스점            쉐이크쉑 청담점            맥도날드 코엑스            맥도날드 청담DT점            노브랜드버거 코엑스            노브랜드버거 역삼역            맥도날드 신사역점            버거킹 강남대로점            맥도날드 삼성DT점            버거킹 선릉역점            버거킹 수서점            맥도날드 압구정CG            버거킹 청담점            맥도날드 선릉점            버거킹 신논현역점            KFC 코엑스Mal            맥도날드 강남 2호            맥도날드 삼성역점            버거킹 차병원사거리            맥도날드 학동역점            맥도날드 한티점            써브웨이 신사역점            노브랜드버거 양재역            버거킹 강남도곡점            노브랜드버거 학동역            롯데리아 역삼점            맥도날드 강남구청점            써브웨이 선릉점            KFC 학동역            롯데리아 수서역사            써브웨이 강남역점            롯데리아 선릉점            버거킹 삼성중앙역점            쉐이크쉑 코엑스점            KFC 강남구청역

      맛있 (49.125)  --        맛있 (29.546)  --        맛있 (29.292)  --       좋아요 (23.251)  --       좋아요 (20.657)  --        맛있 (19.848)  --        맛있 (29.906)  --       좋아요 (14.201)  --       좋아요 (13.995)  --       좋아요 (15.631)  --       버거킹 (13.337)  -

In [ ]:
show_keywords(top_keywords_7, n_fnames_7, restaurant_name_list_7)

       카페 노티드 청담            카페 노티드 삼성            SADDLER HA            아티제 일원점            스타벅스 코엑스몰점            이디야커피랩            테라로사 포스코센터            스타벅스 강남R점            스타벅스 삼성역섬유            도산맘마미아            풀바셋 일원역점            소진담 신사점            엠즈씨드            풀바셋 강남구청역점            C27 가로수길점            바나프레소 역삼점            테라로사 코엑스점            투썸플레이스 신논현            스타벅스 압구정미소            에이비카페 At B            스타벅스 강남비젼타            스타벅스 선릉동신빌            스타벅스 학동사거리            아티제 코엑스몰점            듀자미            브라이언스커피            스타벅스 학동역점            스타벅스 역삼초교사            스템커피            스타벅스 강남오거리            스타벅스 수서역R점            베스킨라빈스 BRO            스타벅스 강남대로점            스타벅스 코엑스사거            스타벅스 양재강남빌            스타벅스 역삼포스코            크리스피크림 수서역            스타벅스 도산가로수            던킨 선릉역점            젠제로            랜디스도넛 가로수길            풀바셋 코엑스몰점            풀바셋 GFC점            스타벅스 논현역사거

      도넛 (61.673)  --        맛있 (42.180)  --        맛있 (36.077)  --      맛있어요 (21.928)  --    

## **불용어 제거 알고리즘** #1. 이모티콘, 영어 제거하는 방법

In [ ]:
def delete_stopwords(key_list): #불용어 제거 함수
  stopwords="아 휴 아이구 아이쿠 아이고 너무 어 나 우리 저희 처음 특히 따라 의해 그냥 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 맛있 맛있어요 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 ㅎㅎ ㅠㅠ ㅅㅅ ㅜㅜ ㅋㅋ "
  stopwords_list=stopwords.split()
  selected_key_list = []
  
  for keywords in key_list:
    selected_list = []
    for word, r in keywords:
        if word in stopwords_list:
            continue
        selected_list.append((word, r))
    selected_key_list.append(selected_list)
  return selected_key_list

In [ ]:
top_keywords_1 = delete_stopwords(top_keywords_1)
top_keywords_2 = delete_stopwords(top_keywords_2)
top_keywords_3 = delete_stopwords(top_keywords_3)
top_keywords_4 = delete_stopwords(top_keywords_4)
top_keywords_5 = delete_stopwords(top_keywords_5)
top_keywords_6 = delete_stopwords(top_keywords_6)
top_keywords_7 = delete_stopwords(top_keywords_7)

In [ ]:
show_keywords(top_keywords_1, n_fnames_1, restaurant_name_list_1)

       은행골 신사점            압구정하루            갓덴스시 강남점            후레쉬빌 삼성역본점            미미면가            샤블리            감탄성신            이끼롤까스 가로수길            상무초밥 강남역점            스시메이진 강남점            미도인 강남            부타이 1막            정돈 강남점            스시마이우 강남CG            홈수끼 학동점            카메스시            샤브쌈주머니 서울강            온기정            카츠바이콘반            청담돈            마코토 청담점            지구당 선릉역점            호호식당 도산공원            스시롭다            이승재참치            올바른스시 강남역점            야스노야 지로 압구            스시마카세 역삼점

      초밥 (18.354)  --       냉모밀 (16.916)  --        초밥 (9.160)  --       돈까스 (14.737)  --        소바 (12.322)  --        친절 (21.249)  --       좋아요 (13.564)  --        친절 (15.054)  --       좋아요 (10.991)  --        초밥 (14.509)  --      스테이크 (10.440)  --        마제 (8.382)  --        안심 (10.967)  --        초밥 (17.639)  --        음식 (18.175)  --        정말 (20.287)  --       가성비 (10.265)  --        텐동 (11.195)  --        진짜 (7.695)  --      카이센동 (13.090)  --        음식 (18.893) 

In [ ]:
show_keywords(top_keywords_2, n_fnames_2, restaurant_name_list_2)

       중앙해장            도쿄등심 압구정점            진미평양냉면            정식당            뱃고동            장인닭갈비 강남점            농민백암순대 본점            탕탕집 논현본점            뽕나무쟁이 선릉본점            새벽집 청담동점            리복집            호족반            도쿄등심 청담점            강남 돼지상회 무한            무월식탁            창고43 강남점            삼육가 본점            류몽민 논현본점            하동관 코엑스몰직영            뱅뱅막국수            미사리밀빛초계국수             농민백암순대 본점            참족            강남 곱            도쿄등심 도곡점            강강술래 역삼점            고기꾼김춘배 강남점            두껍삼 역삼직영점            피양콩할마니            곱창고 강남역점            탐라도야지            백억하누 강남본점            까사생갈비            신둥궁감자탕 선릉직            화기애애            청담동 순도리            온돌 더프라임 강남            무한리필 몽블리 강            수라선            장다리곱창            명랑회관 선릉점            이대성의진면목            명동관            진대감 선릉점            로제닭갈비            남다른감자탕 역삼본            밥밥디라라 역삼본점            갈비다움 역삼본점            램브라튼            창고43 삼성점            송포갈비            이층고깃집 강남구청            수묵당     

In [ ]:
show_keywords(top_keywords_3, n_fnames_3, restaurant_name_list_3)

       울프강 스테이크하우            어글리스토브 신논현            바비레드 강남본점            치즈룸 안다즈            페어링룸            어거스트 힐 강남점            다운타우너 청담            마초쉐프 강남본점            스킬렛 스테이크하우            리틀넥 청담            꽁티드툴레아            크라이치즈버거 삼성            어썸로즈            더플레이스 스타필드            브루클린더버거조인트            메리고라운드 스테이            마녀주방 강남점            텍사스데브라질 압구            클랩피자 청담점            파씨오네            미즈컨테이너 강남             감성타코 강남역점            빌즈 강남            BISTROT de            칙피스 신사점            인터컨티넨탈 코엑스            빠레뜨한남 가로수길            카페마마스 코엑스점            로마옥            을지다락 강남            비어룸            달링키친            코벤트가든 선릉점            마켓오 도곡점            볼트스테이크하우스            노모어피자            오스테리아 꼬또 압            빠레뜨한남 강남CG            미도니셰프 강남점            미도니셰프 삼성점            오스틴            코르바니 선릉            나폴리회관            빠비뚜엡

    스테이크 (61.276)  --        음식 (47.419)  --       파스타 (12.511)  --        음식 (44.681)  --        음식 (51.100)  --       분위기 (48.838)  -- 

In [ ]:
show_keywords(top_keywords_4, n_fnames_4, restaurant_name_list_4)

       아우어베이커리 도산            아우어베이커리 가로            노아베이커리 압구정            파리바게뜨 양재본점            김영모과자점 강남구            김영모과자점 대치점            파리크라상 삼성역키            파리크라상 도곡점            삼송빵집 신논현점            노아베이커리 GFC            비파티세리            타르틴베이커리 도산            아우어베이커리 싸이            파리바게뜨 대청타워            파리바게뜨 신분당선            파리바게뜨 카페선릉            바게트케이            파리바게뜨 압구정역            베이커리로컬 강남역            밀도 한티역점            파리바게뜨 강남역점            김영모과자점 도곡타

      빵이 (8.968)  --       아우어 (6.784)  --       케이크 (37.063)  --       좋아요 (13.100)  --       몽블랑 (6.530)  --        빵이 (6.191)  --       좋아요 (6.523)  --       좋아요 (9.795)  --       좋아요 (8.017)  --       케이크 (23.709)  --       퀸아망 (16.112)  --       좋아요 (12.727)  --       좋아요 (7.376)  --        친절 (3.561)  --       좋아요 (9.168)  --       좋아요 (7.386)  --       바게트 (18.631)  --       좋아요 (10.579)  --       크로플 (8.630)  --        식빵 (8.627)  --       좋아요 (9.265)  --       몽블랑 (8.580)
      커피 (8.654)  --      빨미까레 (6.663)  --      

In [ ]:
show_keywords(top_keywords_5, n_fnames_5, restaurant_name_list_5)

       SMC싱푸차이나            팀호완 삼성점            쮸즈            딤딤섬 파르나스몰점            무탄            라공방 강남1호 본            중경마라탕 강남역점            루위            니뽕내뽕 코엑스점            소각            남소관 우육도삭면            공리            인량훠궈            시추안하우스 삼성점            무한숯불양고치&훠궈            파불라            샤오당쟈마라탕 신사            마라공방 역삼점            미엔아이 천등

      병원 (10.372)  --        딤섬 (14.386)  --       소룡포 (6.453)  --        딤섬 (15.647)  --       멘보샤 (13.371)  --       마라탕 (13.610)  --       마라탕 (13.706)  --        정말 (15.839)  --      니뽕내뽕 (4.939)  --       마라탕 (21.979)  --        진짜 (8.238)  --        공리 (8.830)  --        훠궈 (18.677)  --        마라 (10.279)  --       양꼬치 (21.807)  --        음식 (15.129)  --       마라탕 (12.505)  --       마라탕 (12.540)  --       우육면 (16.345)
      맛은 (4.479)  --        홍콩 (10.016)  --        딤섬 (6.236)  --       웨이팅 (7.280)  --        진짜 (11.435)  --       불친절 (6.009)  --        진짜 (5.836)  --        음식 (15.037)  --        음식 (4.118)  --        진짜 (14.986)  --

In [ ]:
show_keywords(top_keywords_6, n_fnames_6, restaurant_name_list_6)

       쉐이크쉑 강남점            에그슬럿 코엑스점            쉐이크쉑 청담점            맥도날드 코엑스            맥도날드 청담DT점            노브랜드버거 코엑스            노브랜드버거 역삼역            맥도날드 신사역점            버거킹 강남대로점            맥도날드 삼성DT점            버거킹 선릉역점            버거킹 수서점            맥도날드 압구정CG            버거킹 청담점            맥도날드 선릉점            버거킹 신논현역점            KFC 코엑스Mal            맥도날드 강남 2호            맥도날드 삼성역점            버거킹 차병원사거리            맥도날드 학동역점            맥도날드 한티점            써브웨이 신사역점            노브랜드버거 양재역            버거킹 강남도곡점            노브랜드버거 학동역            롯데리아 역삼점            맥도날드 강남구청점            써브웨이 선릉점            KFC 학동역            롯데리아 수서역사            써브웨이 강남역점            롯데리아 선릉점            버거킹 삼성중앙역점            쉐이크쉑 코엑스점            KFC 강남구청역

      쉑쉑 (14.800)  --        맛은 (13.914)  --       좋아요 (10.421)  --       좋아요 (23.251)  --       좋아요 (20.657)  --       가성비 (16.524)  --       좋아요 (18.596)  --       좋아요 (14.201)  --       좋아요 (13.995)  --       좋아요 (15.631)  --       버거킹 (13.337)  -

In [ ]:
show_keywords(top_keywords_7, n_fnames_7, restaurant_name_list_7)

       카페 노티드 청담            카페 노티드 삼성            SADDLER HA            아티제 일원점            스타벅스 코엑스몰점            이디야커피랩            테라로사 포스코센터            스타벅스 강남R점            스타벅스 삼성역섬유            도산맘마미아            풀바셋 일원역점            소진담 신사점            엠즈씨드            풀바셋 강남구청역점            C27 가로수길점            바나프레소 역삼점            테라로사 코엑스점            투썸플레이스 신논현            스타벅스 압구정미소            에이비카페 At B            스타벅스 강남비젼타            스타벅스 선릉동신빌            스타벅스 학동사거리            아티제 코엑스몰점            듀자미            브라이언스커피            스타벅스 학동역점            스타벅스 역삼초교사            스템커피            스타벅스 강남오거리            스타벅스 수서역R점            베스킨라빈스 BRO            스타벅스 강남대로점            스타벅스 코엑스사거            스타벅스 양재강남빌            스타벅스 역삼포스코            크리스피크림 수서역            스타벅스 도산가로수            던킨 선릉역점            젠제로            랜디스도넛 가로수길            풀바셋 코엑스몰점            풀바셋 GFC점            스타벅스 논현역사거

      도넛 (61.673)  --        도넛 (37.673)  --       크로플 (24.376)  --        커피 (8.281)  --     

## **명사 조사 분리 알고리즘** #문장별로 형태소 분석하고, 거기서 'n'로 추출되는 것만 저장? -> 너무 많은 명사들이..

In [ ]:
def get_min_length(list) -> int:
  min_length = 9999999
  for i in range(len(list)):
    if min_length > len(list[i]):
      min_length = len(list[i])
    
  return min_length

In [ ]:
def nounize(list) -> list: # list에 있는 keyword 중 동사를 원형 형태로 바꿔주고, 동일한 원형 Rank는 합해준다.
  noun_list = [[] for i in range(len(list))]
  noun_word_list = [[] for i in range(len(list))]

  han = Hannanum()

  for i in range(len(list)):
    for k in range(len(list[i])):
      mn_word = han.nouns(list[i][k][0])
      # print('원형:'+list[i][k][0], mn_word)
      if mn_word == []:
        if list[i][k][0] in noun_word_list[i]:
          for s in range(len(noun_list)):
            if list[i][k][0] == noun_list[i][s][0]:
              noun_list[i][s][1] += list[i][k][1]
        else:
          noun_list[i].append([list[i][k][0], list[i][k][1]])
          noun_word_list[i].append(list[i][k][0])
      else:
        if mn_word[0] in noun_word_list[i]:
          for n in range(len(noun_list[i])):
            if mn_word[0] == noun_list[i][n][0]:
              noun_list[i][n][1] += list[i][k][1]
        else:
          noun_list[i].append([mn_word[0], list[i][k][1]])
          noun_word_list[i].append(mn_word[0])

###############################################################
# 오름차순 정렬
  sorted_list = []
  for j in range(len(noun_list)):
    sorted_list.append(sorted(noun_list[j], key=lambda x : [x[1]], reverse=1))

  return sorted_list

In [ ]:
top_keywords_1 = nounize(top_keywords_1)
top_keywords_2 = nounize(top_keywords_2)
top_keywords_3 = nounize(top_keywords_3)
top_keywords_4 = nounize(top_keywords_4)
top_keywords_5 = nounize(top_keywords_5)
top_keywords_6 = nounize(top_keywords_6)
top_keywords_7 = nounize(top_keywords_7)

In [ ]:
show_keywords(top_keywords_1, n_fnames_1, restaurant_name_list_1)

       은행골 신사점            압구정하루            갓덴스시 강남점            후레쉬빌 삼성역본점            미미면가            샤블리            감탄성신            이끼롤까스 가로수길            상무초밥 강남역점            스시메이진 강남점            미도인 강남            부타이 1막            정돈 강남점            스시마이우 강남CG            홈수끼 학동점            카메스시            샤브쌈주머니 서울강            온기정            카츠바이콘반            청담돈            마코토 청담점            지구당 선릉역점            호호식당 도산공원            스시롭다            이승재참치            올바른스시 강남역점            야스노야 지로 압구            스시마카세 역삼점

      초밥 (18.354)  --       냉모밀 (16.916)  --        초밥 (9.160)  --       돈까스 (14.737)  --        소바 (12.322)  --        친절 (21.249)  --       좋아요 (13.564)  --        친절 (15.054)  --       좋아요 (10.991)  --        초밥 (14.509)  --      스테이크 (10.440)  --        마제 (8.382)  --        안심 (10.967)  --        초밥 (17.639)  --         맛 (19.864)  --        정말 (20.287)  --         먹 (11.106)  --        텐동 (11.195)  --        진짜 (7.695)  --      카이센동 (13.090)  --        음식 (18.893) 

In [ ]:
show_keywords(top_keywords_2, n_fnames_2, restaurant_name_list_2)

       중앙해장            도쿄등심 압구정점            진미평양냉면            정식당            뱃고동            장인닭갈비 강남점            농민백암순대 본점            탕탕집 논현본점            뽕나무쟁이 선릉본점            새벽집 청담동점            리복집            호족반            도쿄등심 청담점            강남 돼지상회 무한            무월식탁            창고43 강남점            삼육가 본점            류몽민 논현본점            하동관 코엑스몰직영            뱅뱅막국수            미사리밀빛초계국수             농민백암순대 본점            참족            강남 곱            도쿄등심 도곡점            강강술래 역삼점            고기꾼김춘배 강남점            두껍삼 역삼직영점            피양콩할마니            곱창고 강남역점            탐라도야지            백억하누 강남본점            까사생갈비            신둥궁감자탕 선릉직            화기애애            청담동 순도리            온돌 더프라임 강남            무한리필 몽블리 강            수라선            장다리곱창            명랑회관 선릉점            이대성의진면목            명동관            진대감 선릉점            로제닭갈비            남다른감자탕 역삼본            밥밥디라라 역삼본점            갈비다움 역삼본점            램브라튼            창고43 삼성점            송포갈비            이층고깃집 강남구청            수묵당     

In [ ]:
show_keywords(top_keywords_3, n_fnames_3, restaurant_name_list_3)

       울프강 스테이크하우            어글리스토브 신논현            바비레드 강남본점            치즈룸 안다즈            페어링룸            어거스트 힐 강남점            다운타우너 청담            마초쉐프 강남본점            스킬렛 스테이크하우            리틀넥 청담            꽁티드툴레아            크라이치즈버거 삼성            어썸로즈            더플레이스 스타필드            브루클린더버거조인트            메리고라운드 스테이            마녀주방 강남점            텍사스데브라질 압구            클랩피자 청담점            파씨오네            미즈컨테이너 강남             감성타코 강남역점            빌즈 강남            BISTROT de            칙피스 신사점            인터컨티넨탈 코엑스            빠레뜨한남 가로수길            카페마마스 코엑스점            로마옥            을지다락 강남            비어룸            달링키친            코벤트가든 선릉점            마켓오 도곡점            볼트스테이크하우스            노모어피자            오스테리아 꼬또 압            빠레뜨한남 강남CG            미도니셰프 강남점            미도니셰프 삼성점            오스틴            코르바니 선릉            나폴리회관            빠비뚜엡

    스테이크 (61.276)  --        음식 (47.419)  --       파스타 (12.511)  --        음식 (44.681)  --        음식 (51.100)  --       분위기 (48.838)  -- 

In [ ]:
show_keywords(top_keywords_4, n_fnames_4, restaurant_name_list_4)

       아우어베이커리 도산            아우어베이커리 가로            노아베이커리 압구정            파리바게뜨 양재본점            김영모과자점 강남구            김영모과자점 대치점            파리크라상 삼성역키            파리크라상 도곡점            삼송빵집 신논현점            노아베이커리 GFC            비파티세리            타르틴베이커리 도산            아우어베이커리 싸이            파리바게뜨 대청타워            파리바게뜨 신분당선            파리바게뜨 카페선릉            바게트케이            파리바게뜨 압구정역            베이커리로컬 강남역            밀도 한티역점            파리바게뜨 강남역점            김영모과자점 도곡타

       빵 (8.968)  --        아우 (6.784)  --       케이크 (37.063)  --       좋아요 (13.100)  --         빵 (7.616)  --         빵 (8.007)  --       좋아요 (6.523)  --       좋아요 (9.795)  --       좋아요 (8.017)  --       케이크 (23.709)  --       퀸아망 (16.112)  --       좋아요 (12.727)  --       좋아요 (7.376)  --         빵 (3.585)  --       좋아요 (9.168)  --       좋아요 (7.386)  --       바게트 (18.631)  --       좋아요 (10.579)  --       크로플 (8.630)  --        식빵 (8.627)  --       좋아요 (9.265)  --         빵 (10.980)
      커피 (8.654)  --      빨미까레 (6.663)  --     

In [ ]:
show_keywords(top_keywords_5, n_fnames_5, restaurant_name_list_5)

       SMC싱푸차이나            팀호완 삼성점            쮸즈            딤딤섬 파르나스몰점            무탄            라공방 강남1호 본            중경마라탕 강남역점            루위            니뽕내뽕 코엑스점            소각            남소관 우육도삭면            공리            인량훠궈            시추안하우스 삼성점            무한숯불양고치&훠궈            파불라            샤오당쟈마라탕 신사            마라공방 역삼점            미엔아이 천등

      병원 (10.372)  --         맛 (18.055)  --         맛 (7.999)  --        딤섬 (15.647)  --       멘보샤 (13.371)  --       마라탕 (13.610)  --       마라탕 (13.706)  --        정말 (15.839)  --         맛 (11.566)  --       마라탕 (21.979)  --         맛 (8.374)  --        공리 (8.830)  --        훠궈 (18.677)  --         맛 (11.107)  --        꼬치 (27.396)  --        음식 (15.129)  --       마라탕 (12.505)  --       마라탕 (12.540)  --       우육면 (16.345)
       맛 (7.830)  --        딤섬 (14.386)  --       소룡포 (6.453)  --         맛 (12.911)  --         맛 (13.041)  --         맛 (9.567)  --        진짜 (5.836)  --        음식 (15.037)  --      니뽕내뽕 (4.939)  --        진짜 (14.986)  

In [ ]:
show_keywords(top_keywords_6, n_fnames_6, restaurant_name_list_6)

       쉐이크쉑 강남점            에그슬럿 코엑스점            쉐이크쉑 청담점            맥도날드 코엑스            맥도날드 청담DT점            노브랜드버거 코엑스            노브랜드버거 역삼역            맥도날드 신사역점            버거킹 강남대로점            맥도날드 삼성DT점            버거킹 선릉역점            버거킹 수서점            맥도날드 압구정CG            버거킹 청담점            맥도날드 선릉점            버거킹 신논현역점            KFC 코엑스Mal            맥도날드 강남 2호            맥도날드 삼성역점            버거킹 차병원사거리            맥도날드 학동역점            맥도날드 한티점            써브웨이 신사역점            노브랜드버거 양재역            버거킹 강남도곡점            노브랜드버거 학동역            롯데리아 역삼점            맥도날드 강남구청점            써브웨이 선릉점            KFC 학동역            롯데리아 수서역사            써브웨이 강남역점            롯데리아 선릉점            버거킹 삼성중앙역점            쉐이크쉑 코엑스점            KFC 강남구청역

       맛 (18.180)  --         맛 (26.435)  --       좋아요 (10.421)  --       좋아요 (23.251)  --       좋아요 (20.657)  --       가성비 (16.524)  --       좋아요 (18.596)  --       좋아요 (14.201)  --       좋아요 (13.995)  --       좋아요 (15.631)  --       버거킹 (13.337)  -

In [ ]:
show_keywords(top_keywords_7, n_fnames_7, restaurant_name_list_7)

       카페 노티드 청담            카페 노티드 삼성            SADDLER HA            아티제 일원점            스타벅스 코엑스몰점            이디야커피랩            테라로사 포스코센터            스타벅스 강남R점            스타벅스 삼성역섬유            도산맘마미아            풀바셋 일원역점            소진담 신사점            엠즈씨드            풀바셋 강남구청역점            C27 가로수길점            바나프레소 역삼점            테라로사 코엑스점            투썸플레이스 신논현            스타벅스 압구정미소            에이비카페 At B            스타벅스 강남비젼타            스타벅스 선릉동신빌            스타벅스 학동사거리            아티제 코엑스몰점            듀자미            브라이언스커피            스타벅스 학동역점            스타벅스 역삼초교사            스템커피            스타벅스 강남오거리            스타벅스 수서역R점            베스킨라빈스 BRO            스타벅스 강남대로점            스타벅스 코엑스사거            스타벅스 양재강남빌            스타벅스 역삼포스코            크리스피크림 수서역            스타벅스 도산가로수            던킨 선릉역점            젠제로            랜디스도넛 가로수길            풀바셋 코엑스몰점            풀바셋 GFC점            스타벅스 논현역사거

      도넛 (61.673)  --        도넛 (37.673)  --       크로플 (24.376)  --        커피 (8.281)  --     

### **Lemmatizer** **동사 원형 복구**

In [ ]:
%cd korean_lemmatizer-master

/content/drive/MyDrive/nlp_project/korean_lemmatizer-master


In [ ]:
!pip install soylemma

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import soylemma
from soylemma import Lemmatizer

lemmatizer = Lemmatizer()

In [ ]:
def lemmatizer_to_commonkeyword(list, noun_list) -> list: # list에 있는 keyword 중 동사를 원형 형태로 바꿔주고, 동일한 원형 Rank는 합해준다.
  lemmatized_list = [[] for i in range(len(list))]
  lemmatized_word_list = [[] for i in range(len(list))]

  for i in range(len(list)):
    for k in range(len(list[i])):
      mn_word = lemmatizer.lemmatize(list[i][k][0])
      # print('원형:'+list[i][k][0], mn_word)
      if list[i][k][0] in noun_list:
        if list[i][k][0] in lemmatized_word_list[i]:
          for s in range(len(lemmatized_list)):
            if list[i][k][0] == lemmatized_list[i][s][0]:
              lemmatized_list[i][s][1] += list[i][k][1]
        else:
          lemmatized_list[i].append([list[i][k][0], list[i][k][1]])
          lemmatized_word_list[i].append(list[i][k][0])

      elif mn_word == []:
        if list[i][k][0] in lemmatized_word_list[i]:
          for s in range(len(lemmatized_list)):
            if list[i][k][0] == lemmatized_list[i][s][0]:
              lemmatized_list[i][s][1] += list[i][k][1]
        else:
          lemmatized_list[i].append([list[i][k][0], list[i][k][1]])
          lemmatized_word_list[i].append(list[i][k][0])
          
      else:
        if mn_word[0][0] in lemmatized_word_list[i]:
          for n in range(len(lemmatized_list[i])):
            if mn_word[0][0] == lemmatized_list[i][n][0]:
              lemmatized_list[i][n][1] += list[i][k][1]
        else:
          lemmatized_list[i].append([mn_word[0][0], list[i][k][1]])
          lemmatized_word_list[i].append(mn_word[0][0])

######################################################
# 오름차순 정렬
  sorted_list = []
  for j in range(len(lemmatized_list)):
    sorted_list.append(sorted(lemmatized_list[j], key=lambda x : [x[1]], reverse=1))

  return sorted_list

In [ ]:
# def lemmatizer_to_commonkeyword1(list) -> list: # list에 있는 keyword 중 동사를 원형 형태로 바꿔주고, 동일한 원형 Rank는 합해준다.
#   lemmatized_list = [[] for i in range(len(list))]
#   lemmatized_word_list = [[] for i in range(len(list))]

#   for i in range(len(list)):
#     for k in range(len(list[i])):
#       mn_word = lemmatizer.lemmatize(list[i][k][0])
#       # print('원형:'+list[i][k][0], mn_word)
#       if mn_word == []:
#         if list[i][k][0] in lemmatized_word_list[i]:
#           for s in range(len(lemmatized_list)):
#             if list[i][k][0] == lemmatized_list[i][s][0]:
#               lemmatized_list[i][s][1] += list[i][k][1]
#         else:
#           lemmatized_list[i].append([list[i][k][0], list[i][k][1]])
#           lemmatized_word_list[i].append(list[i][k][0])
#       else:
#         if mn_word[0][0] in lemmatized_word_list[i]:
#           for n in range(len(lemmatized_list[i])):
#             if mn_word[0][0] == lemmatized_list[i][n][0]:
#               lemmatized_list[i][n][1] += list[i][k][1]
#         else:
#           lemmatized_list[i].append([mn_word[0][0], list[i][k][1]])
#           lemmatized_word_list[i].append(mn_word[0][0])

# ######################################################
# # 오름차순 정렬
#   sorted_list = []
#   for j in range(len(lemmatized_list)):
#     sorted_list.append(sorted(lemmatized_list[j], key=lambda x : [x[1]], reverse=1))

#   return sorted_list

In [ ]:
top_keywords_1 = lemmatizer_to_commonkeyword(top_keywords_1, noun_list_1)
top_keywords_2 = lemmatizer_to_commonkeyword(top_keywords_2, noun_list_2)
top_keywords_3 = lemmatizer_to_commonkeyword(top_keywords_3, noun_list_3)
top_keywords_4 = lemmatizer_to_commonkeyword(top_keywords_4, noun_list_4)
top_keywords_5 = lemmatizer_to_commonkeyword(top_keywords_5, noun_list_5)
top_keywords_6 = lemmatizer_to_commonkeyword(top_keywords_6, noun_list_6)
top_keywords_7 = lemmatizer_to_commonkeyword(top_keywords_7, noun_list_7)

In [ ]:
show_keywords(top_keywords_1, n_fnames_1, restaurant_name_list_1)

       은행골 신사점            압구정하루            갓덴스시 강남점            후레쉬빌 삼성역본점            미미면가            샤블리            감탄성신            이끼롤까스 가로수길            상무초밥 강남역점            스시메이진 강남점            미도인 강남            부타이 1막            정돈 강남점            스시마이우 강남CG            홈수끼 학동점            카메스시            샤브쌈주머니 서울강            온기정            카츠바이콘반            청담돈            마코토 청담점            지구당 선릉역점            호호식당 도산공원            스시롭다            이승재참치            올바른스시 강남역점            야스노야 지로 압구            스시마카세 역삼점

      초밥 (18.354)  --       냉모밀 (16.916)  --        좋다 (9.948)  --        좋다 (35.078)  --        소바 (12.322)  --        좋다 (22.449)  --        좋다 (31.065)  --        좋다 (25.998)  --        좋다 (25.918)  --        초밥 (14.509)  --        좋다 (13.765)  --        마제 (8.382)  --        안심 (10.967)  --        좋다 (22.974)  --        좋다 (36.724)  --        좋다 (28.568)  --        좋다 (18.469)  --        텐동 (11.195)  --        진짜 (7.695)  --        좋다 (15.214)  --        좋다 (29.883) 

In [ ]:
show_keywords(top_keywords_2, n_fnames_2, restaurant_name_list_2)

       중앙해장            도쿄등심 압구정점            진미평양냉면            정식당            뱃고동            장인닭갈비 강남점            농민백암순대 본점            탕탕집 논현본점            뽕나무쟁이 선릉본점            새벽집 청담동점            리복집            호족반            도쿄등심 청담점            강남 돼지상회 무한            무월식탁            창고43 강남점            삼육가 본점            류몽민 논현본점            하동관 코엑스몰직영            뱅뱅막국수            미사리밀빛초계국수             농민백암순대 본점            참족            강남 곱            도쿄등심 도곡점            강강술래 역삼점            고기꾼김춘배 강남점            두껍삼 역삼직영점            피양콩할마니            곱창고 강남역점            탐라도야지            백억하누 강남본점            까사생갈비            신둥궁감자탕 선릉직            화기애애            청담동 순도리            온돌 더프라임 강남            무한리필 몽블리 강            수라선            장다리곱창            명랑회관 선릉점            이대성의진면목            명동관            진대감 선릉점            로제닭갈비            남다른감자탕 역삼본            밥밥디라라 역삼본점            갈비다움 역삼본점            램브라튼            창고43 삼성점            송포갈비            이층고깃집 강남구청            수묵당     

In [ ]:
show_keywords(top_keywords_3, n_fnames_3, restaurant_name_list_3)

       울프강 스테이크하우            어글리스토브 신논현            바비레드 강남본점            치즈룸 안다즈            페어링룸            어거스트 힐 강남점            다운타우너 청담            마초쉐프 강남본점            스킬렛 스테이크하우            리틀넥 청담            꽁티드툴레아            크라이치즈버거 삼성            어썸로즈            더플레이스 스타필드            브루클린더버거조인트            메리고라운드 스테이            마녀주방 강남점            텍사스데브라질 압구            클랩피자 청담점            파씨오네            미즈컨테이너 강남             감성타코 강남역점            빌즈 강남            BISTROT de            칙피스 신사점            인터컨티넨탈 코엑스            빠레뜨한남 가로수길            카페마마스 코엑스점            로마옥            을지다락 강남            비어룸            달링키친            코벤트가든 선릉점            마켓오 도곡점            볼트스테이크하우스            노모어피자            오스테리아 꼬또 압            빠레뜨한남 강남CG            미도니셰프 강남점            미도니셰프 삼성점            오스틴            코르바니 선릉            나폴리회관            빠비뚜엡

      좋다 (101.393)  --        좋다 (77.725)  --        좋다 (15.799)  --        좋다 (60.321)  --        좋다 (82.878)  --        좋다 (94.628)  --

In [ ]:
show_keywords(top_keywords_4, n_fnames_4, restaurant_name_list_4)

       아우어베이커리 도산            아우어베이커리 가로            노아베이커리 압구정            파리바게뜨 양재본점            김영모과자점 강남구            김영모과자점 대치점            파리크라상 삼성역키            파리크라상 도곡점            삼송빵집 신논현점            노아베이커리 GFC            비파티세리            타르틴베이커리 도산            아우어베이커리 싸이            파리바게뜨 대청타워            파리바게뜨 신분당선            파리바게뜨 카페선릉            바게트케이            파리바게뜨 압구정역            베이커리로컬 강남역            밀도 한티역점            파리바게뜨 강남역점            김영모과자점 도곡타

       빵 (8.968)  --        많다 (7.962)  --       케이크 (37.063)  --        좋다 (17.349)  --         빵 (7.616)  --         빵 (8.007)  --        좋다 (8.831)  --        좋다 (15.268)  --        좋다 (9.775)  --       케이크 (23.709)  --       퀸아망 (16.112)  --        좋다 (23.264)  --        좋다 (11.040)  --         빵 (3.585)  --        좋다 (11.787)  --        좋다 (8.207)  --       바게트 (18.631)  --        좋다 (15.171)  --       크로플 (8.630)  --        식빵 (8.627)  --        좋다 (11.422)  --         빵 (10.980)
      커피 (8.654)  --        좋다 (7.782)  -- 

In [ ]:
show_keywords(top_keywords_5, n_fnames_5, restaurant_name_list_5)

       SMC싱푸차이나            팀호완 삼성점            쮸즈            딤딤섬 파르나스몰점            무탄            라공방 강남1호 본            중경마라탕 강남역점            루위            니뽕내뽕 코엑스점            소각            남소관 우육도삭면            공리            인량훠궈            시추안하우스 삼성점            무한숯불양고치&훠궈            파불라            샤오당쟈마라탕 신사            마라공방 역삼점            미엔아이 천등

      병원 (10.372)  --         맛 (18.055)  --         맛 (7.999)  --        딤섬 (15.647)  --       멘보샤 (13.371)  --       마라탕 (13.610)  --       마라탕 (13.706)  --        좋다 (25.418)  --         맛 (11.566)  --       마라탕 (21.979)  --        먹다 (9.974)  --        공리 (8.830)  --        좋다 (22.710)  --        좋다 (15.189)  --        꼬치 (27.396)  --        좋다 (20.204)  --        좋다 (12.795)  --       마라탕 (12.540)  --       우육면 (16.345)
       맛 (7.830)  --        먹다 (16.331)  --       소룡포 (6.453)  --         맛 (12.911)  --         맛 (13.041)  --         맛 (9.567)  --        진짜 (5.836)  --        정말 (15.839)  --        먹다 (5.291)  --        좋다 (16.316)  

In [ ]:
show_keywords(top_keywords_6, n_fnames_6, restaurant_name_list_6)

       쉐이크쉑 강남점            에그슬럿 코엑스점            쉐이크쉑 청담점            맥도날드 코엑스            맥도날드 청담DT점            노브랜드버거 코엑스            노브랜드버거 역삼역            맥도날드 신사역점            버거킹 강남대로점            맥도날드 삼성DT점            버거킹 선릉역점            버거킹 수서점            맥도날드 압구정CG            버거킹 청담점            맥도날드 선릉점            버거킹 신논현역점            KFC 코엑스Mal            맥도날드 강남 2호            맥도날드 삼성역점            버거킹 차병원사거리            맥도날드 학동역점            맥도날드 한티점            써브웨이 신사역점            노브랜드버거 양재역            버거킹 강남도곡점            노브랜드버거 학동역            롯데리아 역삼점            맥도날드 강남구청점            써브웨이 선릉점            KFC 학동역            롯데리아 수서역사            써브웨이 강남역점            롯데리아 선릉점            버거킹 삼성중앙역점            쉐이크쉑 코엑스점            KFC 강남구청역

       맛 (18.180)  --         맛 (26.435)  --        좋다 (15.519)  --        좋다 (30.276)  --        좋다 (26.216)  --        좋다 (26.673)  --        좋다 (31.149)  --        좋다 (19.158)  --        좋다 (19.596)  --        좋다 (19.153)  --        좋다 (13.357)  -

In [ ]:
show_keywords(top_keywords_7, n_fnames_7, restaurant_name_list_7)

       카페 노티드 청담            카페 노티드 삼성            SADDLER HA            아티제 일원점            스타벅스 코엑스몰점            이디야커피랩            테라로사 포스코센터            스타벅스 강남R점            스타벅스 삼성역섬유            도산맘마미아            풀바셋 일원역점            소진담 신사점            엠즈씨드            풀바셋 강남구청역점            C27 가로수길점            바나프레소 역삼점            테라로사 코엑스점            투썸플레이스 신논현            스타벅스 압구정미소            에이비카페 At B            스타벅스 강남비젼타            스타벅스 선릉동신빌            스타벅스 학동사거리            아티제 코엑스몰점            듀자미            브라이언스커피            스타벅스 학동역점            스타벅스 역삼초교사            스템커피            스타벅스 강남오거리            스타벅스 수서역R점            베스킨라빈스 BRO            스타벅스 강남대로점            스타벅스 코엑스사거            스타벅스 양재강남빌            스타벅스 역삼포스코            크리스피크림 수서역            스타벅스 도산가로수            던킨 선릉역점            젠제로            랜디스도넛 가로수길            풀바셋 코엑스몰점            풀바셋 GFC점            스타벅스 논현역사거

      도넛 (61.673)  --        도넛 (37.673)  --       크로플 (24.376)  --        커피 (8.281)  --     

###**Common keyword 삭제**

In [ ]:
# 공용 키워드 삭제 - 한 대분류 안의 모든 파일 중 70% 이상 중복되는 키워드는 키워드로서 역할을 할 수 없음 ex) 맛있, 많다, 진짜, 좋다, 정말...

def delete_common_keywords(list): 
  new_keyword_counter = {}
  for keywords in list:
    m_words, m_ranks = zip(*keywords)
    for m_word in m_words:
      new_keyword_counter[m_word] = new_keyword_counter.get(m_word, 0) + 1

  new_common_keywords = {m_word for m_word, count in new_keyword_counter.items() if count >= (len(list)*0.7)} # 공용 키워드 설정 기준 = 전체 문서 중 70% 이상 중복인 경우

  str(new_common_keywords)

  print(new_common_keywords)

  new_selected_top_keywords = []
  
  for keywords in list:
    new_selected_keywords = []
    for word, r in keywords:
      if word in new_common_keywords:
          continue
      new_selected_keywords.append((word, r))
    new_selected_top_keywords.append(new_selected_keywords)
  
  return new_selected_top_keywords

In [ ]:
top_keywords_1 = delete_common_keywords(top_keywords_1)
top_keywords_2 = delete_common_keywords(top_keywords_2)
top_keywords_3 = delete_common_keywords(top_keywords_3)
top_keywords_4 = delete_common_keywords(top_keywords_4)
top_keywords_5 = delete_common_keywords(top_keywords_5)
top_keywords_6 = delete_common_keywords(top_keywords_6)
top_keywords_7 = delete_common_keywords(top_keywords_7)

{'만족', '정말', '방문', '좋다', '가성비', '맛집', '먹', '진짜', '많다', '직원', '서비스', '최고', '깔끔', '추천', '엄청', '아주', '맛', '완전', '가격', '역시', '친절', '음식', '먹다', '재방문', '있다', '같다', '가다'}
{'정말', '항상', '방문', '좋다', '맛집', '먹', '진짜', '많다', '자주', '직원', '서비스', '최고', '깔끔', '엄청', '아주', '맛', '가격', '완전', '역시', '친절', '음식', '먹다', '있다', '같다', '가다'}
{'만족', '정말', '항상', '방문', '분위기', '좋다', '먹', '진짜', '많다', '메뉴', '직원', '서비스', '최고', '추천', '엄청', '아주', '매우다', '맛', '가격', '식사', '역시', '친절', '괜찮', '음식', '먹다', '예약', '재방문', '있다', '같다', '가다'}
{'정말', '항상', '좋다', '빵도', '종류', '진짜', '많다', '자주', '직원', '빵은', '최고', '엄청', '매장', '맛', '가격', '친절', '먹다', '빵', '맛나다', '있다'}
{'정말', '항상', '방문', '좋다', '제일', '맛집', '먹', '진짜', '많다', '자주', '최고', '추천', '엄청', '맛', '가격', '완전', '역시', '친절', '음식', '먹다', '있다', '같다', '가다'}
{'정말', '항상', '깨끗', '좋다', '넓고', '빨다', '먹', '진짜', '많다', '자주', '햄버거', '직원', '최고', '엄청', '매장', '맛', '역시', '친절', '주문', '음식', '먹다', '빠르다', '버거', '맛나다', '있다'}
{'정말', '항상', '분위기', '좋다', '넓고', '진짜', '많다', '자주', '직원', '최고', '엄청', '매장', '자리', '맛', '커피', '역시

In [ ]:
show_keywords(top_keywords_1, len(top_keywords_1), restaurant_name_list_1)

       은행골 신사점            압구정하루            갓덴스시 강남점            후레쉬빌 삼성역본점            미미면가            샤블리            감탄성신            이끼롤까스 가로수길            상무초밥 강남역점            스시메이진 강남점            미도인 강남            부타이 1막            정돈 강남점            스시마이우 강남CG            홈수끼 학동점            카메스시            샤브쌈주머니 서울강            온기정            카츠바이콘반            청담돈            마코토 청담점            지구당 선릉역점            호호식당 도산공원            스시롭다            이승재참치            올바른스시 강남역점            야스노야 지로 압구            스시마카세 역삼점

      초밥 (18.354)  --       냉모밀 (16.916)  --        초밥 (9.160)  --       돈까스 (14.737)  --        소바 (12.322)  --       월남쌈 (13.880)  --        후토 (11.126)  --       돈까스 (11.007)  --        초밥 (10.717)  --        초밥 (14.509)  --      스테이크 (10.440)  --        마제 (8.382)  --        안심 (10.967)  --        초밥 (17.639)  --         룸 (17.119)  --        스시 (12.892)  --       월남쌈 (9.470)  --        텐동 (11.195)  --       돈까스 (6.163)  --      카이센동 (13.090)  --       분위기 (11.677)  

In [ ]:
show_keywords(top_keywords_2, len(top_keywords_2), restaurant_name_list_2)

       중앙해장            도쿄등심 압구정점            진미평양냉면            정식당            뱃고동            장인닭갈비 강남점            농민백암순대 본점            탕탕집 논현본점            뽕나무쟁이 선릉본점            새벽집 청담동점            리복집            호족반            도쿄등심 청담점            강남 돼지상회 무한            무월식탁            창고43 강남점            삼육가 본점            류몽민 논현본점            하동관 코엑스몰직영            뱅뱅막국수            미사리밀빛초계국수             농민백암순대 본점            참족            강남 곱            도쿄등심 도곡점            강강술래 역삼점            고기꾼김춘배 강남점            두껍삼 역삼직영점            피양콩할마니            곱창고 강남역점            탐라도야지            백억하누 강남본점            까사생갈비            신둥궁감자탕 선릉직            화기애애            청담동 순도리            온돌 더프라임 강남            무한리필 몽블리 강            수라선            장다리곱창            명랑회관 선릉점            이대성의진면목            명동관            진대감 선릉점            로제닭갈비            남다른감자탕 역삼본            밥밥디라라 역삼본점            갈비다움 역삼본점            램브라튼            창고43 삼성점            송포갈비            이층고깃집 강남구청            수묵당     

In [ ]:
show_keywords(top_keywords_3, len(top_keywords_3), restaurant_name_list_3)

       울프강 스테이크하우            어글리스토브 신논현            바비레드 강남본점            치즈룸 안다즈            페어링룸            어거스트 힐 강남점            다운타우너 청담            마초쉐프 강남본점            스킬렛 스테이크하우            리틀넥 청담            꽁티드툴레아            크라이치즈버거 삼성            어썸로즈            더플레이스 스타필드            브루클린더버거조인트            메리고라운드 스테이            마녀주방 강남점            텍사스데브라질 압구            클랩피자 청담점            파씨오네            미즈컨테이너 강남             감성타코 강남역점            빌즈 강남            BISTROT de            칙피스 신사점            인터컨티넨탈 코엑스            빠레뜨한남 가로수길            카페마마스 코엑스점            로마옥            을지다락 강남            비어룸            달링키친            코벤트가든 선릉점            마켓오 도곡점            볼트스테이크하우스            노모어피자            오스테리아 꼬또 압            빠레뜨한남 강남CG            미도니셰프 강남점            미도니셰프 삼성점            오스틴            코르바니 선릉            나폴리회관            빠비뚜엡

    스테이크 (61.276)  --        자다 (14.891)  --       파스타 (12.511)  --        치즈 (30.234)  --       파스타 (17.902)  --      스테이크 (25.962)  -- 

In [ ]:
show_keywords(top_keywords_4, len(top_keywords_4), restaurant_name_list_4)

       아우어베이커리 도산            아우어베이커리 가로            노아베이커리 압구정            파리바게뜨 양재본점            김영모과자점 강남구            김영모과자점 대치점            파리크라상 삼성역키            파리크라상 도곡점            삼송빵집 신논현점            노아베이커리 GFC            비파티세리            타르틴베이커리 도산            아우어베이커리 싸이            파리바게뜨 대청타워            파리바게뜨 신분당선            파리바게뜨 카페선릉            바게트케이            파리바게뜨 압구정역            베이커리로컬 강남역            밀도 한티역점            파리바게뜨 강남역점            김영모과자점 도곡타

      커피 (8.654)  --        아우 (6.784)  --       케이크 (37.063)  --        다양 (5.428)  --        몽블 (6.530)  --       김영모 (5.862)  --       비싸다 (4.168)  --        만족 (3.935)  --      옥수수빵 (6.088)  --       케이크 (23.709)  --       퀸아망 (16.112)  --       분위기 (6.554)  --        아우 (4.587)  --       편하다 (2.302)  --        사다 (2.081)  --       샐러드 (2.345)  --       바게트 (18.631)  --        넓고 (2.534)  --       크로플 (8.630)  --        식빵 (8.627)  --        넓고 (3.280)  --        몽블 (8.580)
    빨미까레 (8.025)  --      빨미까레 (6.663)  --       얼그

In [ ]:
show_keywords(top_keywords_5, len(top_keywords_5), restaurant_name_list_5)

       SMC싱푸차이나            팀호완 삼성점            쮸즈            딤딤섬 파르나스몰점            무탄            라공방 강남1호 본            중경마라탕 강남역점            루위            니뽕내뽕 코엑스점            소각            남소관 우육도삭면            공리            인량훠궈            시추안하우스 삼성점            무한숯불양고치&훠궈            파불라            샤오당쟈마라탕 신사            마라공방 역삼점            미엔아이 천등

      병원 (10.372)  --        딤섬 (14.386)  --       소룡포 (6.453)  --        딤섬 (15.647)  --       멘보샤 (13.371)  --       마라탕 (13.610)  --       마라탕 (13.706)  --       분위기 (11.075)  --      니뽕내뽕 (4.939)  --       마라탕 (21.979)  --        우육 (8.202)  --        공리 (8.830)  --        훠궈 (18.677)  --        말다 (10.279)  --        꼬치 (27.396)  --        코스 (11.213)  --       마라탕 (12.505)  --       마라탕 (12.540)  --       우육면 (16.345)
      식당 (3.982)  --        홍콩 (10.016)  --        딤섬 (6.236)  --       웨이팅 (7.280)  --       짜장면 (9.897)  --       불친절 (6.009)  --      꿔바로우 (4.489)  --       서비스 (8.829)  --       테이블 (3.588)  --      꿔바로우 (10.896)  --  

In [ ]:
show_keywords(top_keywords_6, len(top_keywords_6), restaurant_name_list_6)

       쉐이크쉑 강남점            에그슬럿 코엑스점            쉐이크쉑 청담점            맥도날드 코엑스            맥도날드 청담DT점            노브랜드버거 코엑스            노브랜드버거 역삼역            맥도날드 신사역점            버거킹 강남대로점            맥도날드 삼성DT점            버거킹 선릉역점            버거킹 수서점            맥도날드 압구정CG            버거킹 청담점            맥도날드 선릉점            버거킹 신논현역점            KFC 코엑스Mal            맥도날드 강남 2호            맥도날드 삼성역점            버거킹 차병원사거리            맥도날드 학동역점            맥도날드 한티점            써브웨이 신사역점            노브랜드버거 양재역            버거킹 강남도곡점            노브랜드버거 학동역            롯데리아 역삼점            맥도날드 강남구청점            써브웨이 선릉점            KFC 학동역            롯데리아 수서역사            써브웨이 강남역점            롯데리아 선릉점            버거킹 삼성중앙역점            쉐이크쉑 코엑스점            KFC 강남구청역

      쉑쉑 (14.800)  --        슬럿 (10.727)  --        쉑쉑 (9.942)  --        저렴 (8.259)  --       편하다 (8.420)  --       가성비 (16.524)  --       가성비 (17.263)  --      맥도날드 (6.809)  --       버거킹 (13.636)  --      맥도날드 (6.185)  --       버거킹 (13.337)  --    

In [ ]:
show_keywords(top_keywords_7, len(top_keywords_7), restaurant_name_list_7)

       카페 노티드 청담            카페 노티드 삼성            SADDLER HA            아티제 일원점            스타벅스 코엑스몰점            이디야커피랩            테라로사 포스코센터            스타벅스 강남R점            스타벅스 삼성역섬유            도산맘마미아            풀바셋 일원역점            소진담 신사점            엠즈씨드            풀바셋 강남구청역점            C27 가로수길점            바나프레소 역삼점            테라로사 코엑스점            투썸플레이스 신논현            스타벅스 압구정미소            에이비카페 At B            스타벅스 강남비젼타            스타벅스 선릉동신빌            스타벅스 학동사거리            아티제 코엑스몰점            듀자미            브라이언스커피            스타벅스 학동역점            스타벅스 역삼초교사            스템커피            스타벅스 강남오거리            스타벅스 수서역R점            베스킨라빈스 BRO            스타벅스 강남대로점            스타벅스 코엑스사거            스타벅스 양재강남빌            스타벅스 역삼포스코            크리스피크림 수서역            스타벅스 도산가로수            던킨 선릉역점            젠제로            랜디스도넛 가로수길            풀바셋 코엑스몰점            풀바셋 GFC점            스타벅스 논현역사거

      도넛 (61.673)  --        도넛 (37.673)  --       크로플 (24.376)  --         빵 (6.826)  --     

### **키워드 추출 및 Excel파일로 변환**

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def ten_keywords_extractor(empty_dictionary, keywords_list, restaurant_name) -> dict:
  for i in range(len(keywords_list)):
    empty_dictionary[restaurant_name[i]] = [keywords_list[i][k][0] for k in range(10)] 
  
  return empty_dictionary

def show_ten_keywords(dict, list, restaurant_name) -> None:
  for j in range(len(list)):
    print(restaurant_name[j], ':' ,dict[restaurant_name[j]])

def makeall_excelfile(dict, restaurant_name, restaurant_type):
  df = pd.DataFrame(dict)
  df.set_index(restaurant_name)

  df.to_excel("./"+restaurant_type+".xlsx")

In [ ]:
final_keywords_dict_1, final_keywords_dict_2, final_keywords_dict_3, final_keywords_dict_4, final_keywords_dict_5, final_keywords_dict_6, final_keywords_dict_7 = {}, {}, {}, {}, {}, {}, {}

In [ ]:
final_keywords_dict_1 = ten_keywords_extractor(final_keywords_dict_1, top_keywords_1, restaurant_name_list_1)
final_keywords_dict_2 = ten_keywords_extractor(final_keywords_dict_2, top_keywords_2, restaurant_name_list_2)
final_keywords_dict_3 = ten_keywords_extractor(final_keywords_dict_3, top_keywords_3, restaurant_name_list_3)
final_keywords_dict_4 = ten_keywords_extractor(final_keywords_dict_4, top_keywords_4, restaurant_name_list_4)
final_keywords_dict_5 = ten_keywords_extractor(final_keywords_dict_5, top_keywords_5, restaurant_name_list_5)
final_keywords_dict_6 = ten_keywords_extractor(final_keywords_dict_6, top_keywords_6, restaurant_name_list_6)
final_keywords_dict_7 = ten_keywords_extractor(final_keywords_dict_7, top_keywords_7, restaurant_name_list_7)

In [ ]:
# show_ten_keywords(final_keywords_dict_1, top_keywords_1, restaurant_name_list_1)

In [ ]:
# show_ten_keywords(final_keywords_dict_2, top_keywords_2, restaurant_name_list_2)

In [ ]:
# show_ten_keywords(final_keywords_dict_3, top_keywords_3, restaurant_name_list_3)

In [ ]:
# show_ten_keywords(final_keywords_dict_4, top_keywords_4, restaurant_name_list_4)

In [ ]:
# show_ten_keywords(final_keywords_dict_5, top_keywords_5, restaurant_name_list_5)

In [ ]:
# show_ten_keywords(final_keywords_dict_6, top_keywords_6, restaurant_name_list_6)

In [ ]:
# show_ten_keywords(final_keywords_dict_7, top_keywords_7, restaurant_name_list_7)

In [ ]:
!pwd

/content/drive/MyDrive/nlp_project/korean_lemmatizer-master


In [ ]:
cd /content/drive/MyDrive/nlp_project/키워드_결과물_전체

/content/drive/MyDrive/nlp_project/키워드_결과물_전체


In [ ]:
makeall_excelfile(final_keywords_dict_1, restaurant_name_list_1, restaurant_type[0])
makeall_excelfile(final_keywords_dict_2, restaurant_name_list_2, restaurant_type[1])
makeall_excelfile(final_keywords_dict_3, restaurant_name_list_3, restaurant_type[2])
makeall_excelfile(final_keywords_dict_4, restaurant_name_list_4, restaurant_type[3])
makeall_excelfile(final_keywords_dict_5, restaurant_name_list_5, restaurant_type[4])
makeall_excelfile(final_keywords_dict_6, restaurant_name_list_6, restaurant_type[5])
makeall_excelfile(final_keywords_dict_7, restaurant_name_list_7, restaurant_type[6])

### **상위 50% 키워드 추출**

In [ ]:
!pwd

/content/drive/MyDrive/nlp_project/키워드_결과물_전체


In [ ]:
cd /content/drive/MyDrive/nlp_project/키워드_결과물_상위50

/content/drive/MyDrive/nlp_project/키워드_결과물_상위50


In [ ]:
def fifty_finder(diction, name_list):
  counts = {}
  tmp_list = []

  for name in name_list: 
    for i in range(len(diction[name])):
      tmp_list.append(diction[name][i])

  for x in tmp_list:
    if x in counts:
      counts[x] += 1
    else:
      counts[x] = 1

  counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
  
  return counts

In [ ]:
def make_50excelfile(dict, restaurant_type):
  df = pd.DataFrame(dict)
  df.to_excel("./"+restaurant_type+"_상위50%키워드.xlsx")

In [ ]:
final_keywords_dict_1 = fifty_finder(final_keywords_dict_1, restaurant_name_list_1)
final_keywords_dict_2 = fifty_finder(final_keywords_dict_2, restaurant_name_list_2)
final_keywords_dict_3 = fifty_finder(final_keywords_dict_3, restaurant_name_list_3)
final_keywords_dict_4 = fifty_finder(final_keywords_dict_4, restaurant_name_list_4)
final_keywords_dict_5 = fifty_finder(final_keywords_dict_5, restaurant_name_list_5)
final_keywords_dict_6 = fifty_finder(final_keywords_dict_6, restaurant_name_list_6)
final_keywords_dict_7 = fifty_finder(final_keywords_dict_7, restaurant_name_list_7)

In [ ]:
make_50excelfile(final_keywords_dict_1, restaurant_type[0])
make_50excelfile(final_keywords_dict_2, restaurant_type[1])
make_50excelfile(final_keywords_dict_3, restaurant_type[2])
make_50excelfile(final_keywords_dict_4, restaurant_type[3])
make_50excelfile(final_keywords_dict_5, restaurant_type[4])
make_50excelfile(final_keywords_dict_6, restaurant_type[5])
make_50excelfile(final_keywords_dict_7, restaurant_type[6])

------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# final_keywords_dict_2 = fifty_finder(final_keywords_dict_2, restaurant_name_list_2)
# print(final_keywords_dict_2)
# df = pd.DataFrame(final_keywords_dict_2)

# df.to_excel("./한식_상위50%키워드_다른버전.xlsx")

In [ ]:
# df = pd.DataFrame(final_keywords_dict_2)
# df.set_index(restaurant_name_list_2)

# # df.to_excel("./한식.xlsx")

In [ ]:
# keywords_list_2 = {}
# for i in range(len(top_keywords_2)):
#   keywords_list_2[restaurant_name_list_2[i]] = [top_keywords_2[i][k][0] for k in range(10)] 

# for j in range(len(top_keywords_2)):
#   print(restaurant_name_list_2[j], ':' ,keywords_list_2[restaurant_name_list_2[j]])

In [ ]:
# keyword_counter_4 = {}
# for keywords in top_keywords_4:
#   words, ranks = zip(*keywords)
#   for word in words:
#     keyword_counter_4[word] = keyword_counter_4.get(word, 0) + 1

# common_keywords_4 = {word for word, count in keyword_counter_4.items() if count >= (len(top_keywords_4)*0.7)} # 공용 키워드 설정 기준 = 전체 문서 중 70% 이상 중복인 경우

# str(common_keywords_4)

# print(common_keywords_4)

# selected_top_keywords_4 = []

# for keywords in top_keywords_4:
#   selected_keywords_4 = []
#   for word, r in keywords:
#     if word in common_keywords_4:
#         continue
#     selected_keywords_4.append((word, r))
#   selected_top_keywords_4.append(selected_keywords_4)

In [ ]:
# def fifty_finder(list):
#   real_keyword_counter = {}
#   for keywords in list:
#     words, ranks = zip(*keywords)
#     for word in words:
#       real_keyword_counter[word] = real_keyword_counter.get(word, 0) + 1

#   real_keyword_counter = sorted(real_keyword_counter.items(), key = lambda x:x[1], reverse=True)

#   return real_keyword_counter